In [ ]:
 # what I want to happen:

# animal.trigger_func = TriggerFunction(*[trigger_data])

# animal.trigger_func(own_team, opp_team) --> number of triggers for this ability

In [ ]:
import numpy as np

In [1]:
# team_conds for triggers by player:
def Has2Gold(owner, shop, in_team1):
    return shop.gold >= 2

def Has3Gold(owner, shop, in_team1):
    return shop.gold >= 3

def HasLvl3Pet(owner, shop, in_team1):
    return any([pet.lvl == 3 for pet in shop.team1])

def NotFullTeam(owner, shop, in_team1):
    return len(shop.team1) < Team.max_size

# team_conds for "battle" triggers (non-player triggers)
def IsInShop(owner, gamestate, in_team1):
    return not isinstance(gamestate, Battle)

def LostLastBattle(owner, gamestate, in_team1):
    if isinstance(gamestate, Battle):
        return False
    return gamestate.lost_last_battle

def OwnerIsHighestTier(owner, gamestate, in_team1):
    own_team = gamestate.team1 if in_team1 else gamestate.team2
    return all([pet.lvl < owner.lvl for pet in own_team.exclude(owner.pos)])

# pet_conds
def HasStartOfBattle(pet):
    return any([abl.trigger == 'StartOfBattle' for abl in pet.abilities[pet.lvl]])

def HasSellAbility(pet):
    return any([abl.trigger == 'Sell' for abl in pet.abilities[pet.lvl]])

def NotAlpaca(pet):
    return pet.species != 'Alpaca'
    
def NotWhiteTiger(pet):
    return pet.species != 'White Tiger'

In [2]:
class TriggerFunction:
    def __init__(self, owner, trigger, bywhom, trigger_cond, get_triggerer):
        self.owner = owner
        self.trigger = trigger
        self.bywhom = bywhom
        self.name = trigger
        if bywhom not in ['NoActor', 'Player']:
            self.name += f' {bywhom}'
        self.team_cond = trigger_cond.get('team_cond_func', lambda *args: True)
        self.pet_cond = trigger_cond.get('pet_cond_func', lambda *args: True)
        self.get_triggerer = get_triggerer
            
        if bywhom == 'Enemy':
            def function(self, own_team, opp_team):
                return self.parse_for_triggers(opp_team.pets)
            self.function = function
            
        elif bywhom == 'Friend':
            def function(self, own_team, opp_team):
                return self.parse_for_triggers(own_team.exclude(self.owner.pos))
            self.function = function
        
        elif bywhom == 'FriendAhead':
            def function(self, own_team, opp_team):
                return self.parse_for_triggers([own_team[self.owner.pos - 1]])
            self.function = function
        
        elif bywhom == 'Self':
            def function(self, own_team, opp_team):
                return self.parse_for_triggers([self.owner])
            self.function = function

        elif bywhom == 'TeamPet':
            def function(self, own_team, opp_team):
                return self.parse_for_triggers(own_team)
            self.function = function
    
    def __call__(self, gamestate, in_team1, get_triggerer):
        if self.bywhom == 'Player' or self.trigger == 'NoTrigger':
            return (0, [])
        if not self.team_cond(self.owner, gamestate, in_team1):
            return (0, [])
        else:
            own_team = gamestate.team1 if in_team1 else gamestate.team2
            opp_team = gamestate.team2 if in_team1 else gamestate.team1
            return self.function(self, own_team, opp_team)
        
    def parse_for_triggers(self, pet_list):
        if self.get_triggerer:
            triggerers = []
            for pet in pet_list:
                if self.pet_cond(pet) and pet.trigger_dict[self.trigger] > 0:
                    triggerers.append((pet, pet.trigger_dict[self.trigger]))
            return (1 if len(triggerers) > 0 else 0, triggerers)
        else:
            return (sum([pet.trigger_dict[self.trigger] if self.pet_cond(pet) else 0 for pet in pet_list]), [])
    


Animal Actions:
* Faint
* Sell
* Eats
* Transformed (butterfly)
* Level up
* Summoned 
* Pushed (enemy only)
* Buy
* before attack
* before faint
* hurt
* eats apple
* summoned outside battle (lobster)
* Aura
* Clash (added)


Actors:
* Self
* Friend
* Enemy
* Friend ahead
* Any teammate (including Self)


Player Actions:
* End turn
* Start of turn
* Start of battle
* end of turn
* buy shop food
* upgrade shop tier
* roll
* buy tier 1 pet
